In [ ]:
# Extracted and Inserted JSON data to MongoDB(COVID Dataset)

# Reading covid_cases
covid_cases_url = "https://data.cityofnewyork.us/api/views/rc75-m7u3/rows.json?accessType=DOWNLOAD"
covid_cases_response = urllib.request.urlopen(covid_cases_url)
covid_cases_data = json.loads(covid_cases_response.read())

# inserting into mongoDB
covid_cases_collection = project_db["covid_cases"]
# deleting extising data
covid_cases_collection.drop()


covid_cases_cols = list(filter(lambda val: val["id"] != -1, covid_cases_data["meta"]["view"]["columns"]))
covid_cases_cols_others_len = len(list(filter(lambda val: val["id"] == -1, covid_cases_data["meta"]["view"]["columns"])))


for row in covid_cases_data["data"]:
    db_row = {}
    for col in covid_cases_cols:
        db_row[col["fieldName"].lower()] = row[int(col["position"]) + covid_cases_cols_others_len - 1]
    covid_cases_collection.insert_one(db_row)

print("Inserted air_quality document count: ",covid_cases_collection.count_documents({}))

In [ ]:
##inserted values in covid table

covid_cases_collection = project_db["covid_cases"]

cursor = conn.cursor()
for row in covid_cases_collection.find():
    date = row["date_of_interest"]
    cursor.execute("INSERT INTO covid_cases (date_of_interest, case_count) VALUES (%s,%s)", 
                   (date, row["case_count"]))
cursor.close()
conn.commit()

In [ ]:
#Plots using Matplot library(4th plot)

fig, ax1 = plt.subplots(figsize=(9, 6))
ax2 = ax1.twinx()  
ax2.set_ylim(4, 20);

COLOR_UNEMPLOYMENT = "#3399e6"
COLOR_AIR_QUALITY = "#69b3a2"

#left side y axis
ax1.bar(all_data["Date"], all_data["Unemployment Rate"], color=COLOR_UNEMPLOYMENT) 
ax1.set_ylabel("Unemployment Rate", color=COLOR_UNEMPLOYMENT, fontsize=14)
ax1.tick_params(axis="y", labelcolor=COLOR_UNEMPLOYMENT)
x_dates = unemployment_rate_df['Date'].dt.strftime('%Y-%m-%d').sort_values().unique()
arr = ax1.set_xticklabels(labels=x_dates, rotation=45, ha='right')

#right side y axis
ax2.scatter(all_data["Date"], all_data["Air Quality Data Value"], 
            s = all_data["Unemployment Rate"], color=COLOR_AIR_QUALITY) 
ax2.set_ylabel("Air Quality Data Value", color=COLOR_AIR_QUALITY, fontsize=14)
ax2.tick_params(axis="y", labelcolor=COLOR_AIR_QUALITY)